In [ ]:
import torch

from collections import deque

import numpy as np

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import gym

In [ ]:
!pip install wandb

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: amanichopra. Use `wandb login --relogin` to force relogin


True

## Policy Gradient

In the previous lab, we talked about value based method for reinforcement learning. In this lab, we focus on policy based method.

In policy based methods, intead of defining a value function $Q_\theta(s,a)$ and inducing a policy based on argmax, we parameterize a stochastic policy directly. The policy is parameterized as a categorical distribution over actions. Let it be $\pi_\phi(s)$ with parameter $\phi$, then the policy is defined by sampling actions $$a \sim \pi_\phi(s)$$

The policy induces a probability $p(\tau)$ over trajectories $\tau = \{s_0,a_0,s_1,a_1,..\}$. The expected total discounted reward is

$$\rho(\phi) = \mathbb{E}_{\tau \sim p(\tau)} \big[R(\tau)\big] = \mathbb{E}_{\pi_\phi} \big[\sum_{t=0}^\infty r_t \gamma^t \big]$$

The aim is to find $\phi$ such that the expected reward induced by $\pi_\phi$ is maximized.

### Policy Gradient Computation

We can derive policy gradient

$$\nabla_\phi \rho(\phi) = \mathbb{E}_{\pi} \big[\sum_{t=0}^\infty Q^{\pi_\phi}(s_t,a_t) \nabla_\phi \log \pi_\phi(s_t, a_t) \big]$$

To compute the gradient for update $\phi \leftarrow \phi + \alpha \nabla_\phi \rho(\phi)$, we need to estimate $Q^{\pi_\phi}(s,a)$. Since $Q^{\pi_\phi}(s,a)$ is usually not analytically accessible, it can be approximated by
1. Monte Carlo estimate
2. Train a value function $Q_\theta(s,a) \approx Q^{\pi_\phi}(s,a)$ and use it as a proxy
3. Mixture of both above

Before estimating $Q^{\pi_\phi}(s,a)$, let us write a parameterized policy over actions. The policy $\pi_\phi(s)$ takes a state as input and outputs a categorical distribution over actions. For example, if we have two actions, the probability vector to output is of the form $\pi_\phi(s)=[0.6,0.4]$.

**Loss function**
Given samples of state action pairs $(s_i, a_i)$ and estimate $Q_i$ for $Q^{\pi_\phi}(s_i, a_i)$, for $i=1,\ldots, $ we  set the loss function  as
$$-\frac{1}{N} \sum_{i=1}^N Q_i \log(\pi_\phi(s_i, a_i)) $$
The loss function enables us  to compute policy gradients in implementation. The negative gradient of the above loss function has the form

$$\frac{1}{N} \sum_{i=1}^N Q_i  \nabla_\phi \log \pi_\phi(s_i, a_i) $$

where $Q_i$s are estimated and $\nabla_\phi \log\pi_\phi(s_i, a_i)$s are computed via backprop.

In [ ]:
class PNet(torch.nn.Module):
    def __init__(self, observation_space, action_space):
        super(PNet, self).__init__()
        self.l1 = torch.nn.Linear(observation_space, 128)
        self.l2 = torch.nn.Linear(128, action_space)

    def forward(self, x):
        x = self.l1(x)
        x = torch.nn.functional.relu(x)
        actions = self.l2(x)
        action_probs = torch.nn.functional.softmax(actions, dim=1)
        return action_probs

# define neural net \pi_\phi(s) as a class
class Policy(object):

    def __init__(self, obssize, actsize, lr, device):
        """
        obssize: size of the states
        actsize: size of the actions
        """
        # TODO DEFINE THE MODEL
        self.model = PNet(obssize, actsize).to(device)

        # DEFINE THE OPTIMIZER
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        # RECORD HYPER-PARAMS
        self.obssize = obssize
        self.actsize = actsize
        self.device = device

        # TEST
        #self.compute_prob(np.random.randn(obssize).reshape(1, -1))

    def compute_prob(self, states):
        """
        compute prob distribution over all actions given state: pi(s)
        states: numpy array of size [numsamples, obssize]
        return: numpy array of size [numsamples, actsize]
        """
        states = torch.from_numpy(states).float().unsqueeze(0).to(self.device)
        return self.model(states)

    def _to_one_hot(self, y, num_classes):
        """
        convert an integer vector y into one-hot representation
        """
        scatter_dim = len(y.size())
        y_tensor = y.view(*y.size(), -1)
        zeros = torch.zeros(*y.size(), num_classes, dtype=y.dtype).to(self.device)
        return zeros.scatter(scatter_dim, y_tensor, 1)


    def train(self, ads, lps):
        policy_loss = []

        for d, lp in zip(ads, lps):
            # gradient ascent
            policy_loss.append(-d * lp)
        self.optimizer.zero_grad()
        sum(policy_loss).backward()
        self.optimizer.step()

Try to rollout trajecories using the policy

In [ ]:
# Below is a set of template code for running a policy to interact with the environment
# It initializes a policy and runs it
# Note that you may not be able to run the code properly if there still some undefined components on the Policy class

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v0")
obssize = env.observation_space.low.size
actsize = env.action_space.n
policyinit=Policy(obssize, actsize, 0.1, device)

obs = env.reset()
done = False
while not done:
    prob = policyinit.compute_prob(obs).cpu().data.numpy().ravel()
    prob /= np.sum(prob) #normalizing again to account for numerical errors
    action = np.random.choice(actsize, p=prob.flatten(), size=1)[0] #choose according distribution prob
    obs, reward, done, info = env.step(action)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


### Estimate $Q^\pi(s,a)$

To estimate $Q^\pi(s,a)$, we can rollout the policy until the episode ends and do monte carlo estimate. In particular, under policy $\pi$, we start from state action $(s_0,a_0)$ and rollout the policy to generate a trajectory $\{s_0,a_0,s_1,a_1...s_T,a_T\}$, with corresponding reward $r_0,r_1...r_T$. Monte carlo estimate is

$$\hat{Q}_{MC}(s,a) = \sum_{t=0}^T r_t \gamma^t \approx Q^\pi(s,a)$$

This estimate by itself is of high variance. Using pure monte carlo estimate may work but the gradient can have large variance and hence take the algorithm  a long time to converge. We can reduce variance using baseline. Recall the derivation of PG

$$\nabla_\phi \rho(\phi) = \mathbb{E}_{\pi_\phi} \big[\sum_{t=0}^\infty Q^{\pi_\phi}(s_t,a_t) \nabla_\phi \log \pi_\phi(s_t, a_t) \big] = \mathbb{E}_{\pi_\phi} \big[\sum_{t=0}^\infty ( Q^{\pi_\phi}(s_t,a_t) - b(s_t)) \nabla_\phi \log \pi_\phi(s_t, a_t) \big]$$

where $b(s_t)$ can be any function of state $s_t$. $b(s_t)$ is called baseline. Optimal baseline function is hard to compute, but a good proxy is the value function $V^{\pi_\phi}(s_t)$. Hence the gradient has the form
$$\nabla_\phi \rho(\pi_{\phi}) = \mathbb{E}_{\pi} \big[\sum_{t=0}^\infty A^{\pi_\phi}(s_t,a_t) \nabla_\phi \log \pi_\phi(s_t, a_t) \big]$$

where $A^{\pi_\phi}(s,a)$ is the advantage. Hence we can train a value function $V^{\pi_\phi}(s)$ along side the policy and use it as baseline to reduce the variance of PG.

Hence we also parameterize a value function $V_\theta(s) \approx V^{\pi_\phi}(s)$ with parameter $\theta$ to serve as baseline. The function takes as input the states $s$ and outputs a real value.

Notice that unlike DQN, where $Q_\theta(s,a) \approx Q^\ast(s,a)$, now we have $V_\theta(s) \approx V^{\pi_\phi}(s)$. Therefore, we have a moving target to approximate, that changes with the current policy $\pi_\phi$. As $\phi$ is updated by PG, $\pi_\phi$ keeps changing, which changes $V^{\pi_\phi}(s)$ as well. We need to adapt $V_\theta(s)$ online to cater for the change in policy.

Recall that to evaluate a policy $\pi$, we collect rollouts using $\pi$. If we start with state $s_0$, the reward following $\pi$ thereafter is $r_0...r_{T}$  then

$$V^\pi(s_0) \approx \sum_{t=0}^{T} r_t \gamma^{t} = \hat{V}(s_0)$$

In general, given a trajectory $(s_0, a_0, s_1, a_1, r_1, s_2, a_2, r_2, ..., s_{T+1})$

$$\hat{V}(s_i) = \sum_{i=t}^{T} r_i \gamma^{i-t}$$

And the objective to minimize over is
$$\frac{1}{T+1} \sum_{i=0}^{T} (V_\theta(s_i) - \hat{V}(s_i))^2$$

Since the policy keeps updating, we do not need to minimize the above objective to optimality. In practice, taking one gradient step w.r.t. above objective suffices.

In the code cell below, define the neural network to learn value function estimate. The implementation is similar to Qfunction class in lab3, except that inputs are only states, and not actions.

In [ ]:
class VNet(torch.nn.Module):
    def __init__(self, observation_space):
        super(VNet, self).__init__()
        self.l1 = torch.nn.Linear(observation_space, 128)
        self.l2 = torch.nn.Linear(128, 1)

    def forward(self, x):
        x = self.l1(x)
        x = torch.nn.functional.relu(x)
        state_value = self.l2(x)
        return state_value

# TODO: define value function as a class. You need to define the model and set the loss.

class ValueFunction(object):
    def __init__(self, obssize, lr, device):
        """
        obssize: size of states
        """
        # TODO DEFINE THE MODEL
        self.model = VNet(obssize).to(device)

        # DEFINE THE OPTIMIZER
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        # RECORD HYPER-PARAMS
        self.obssize = obssize
        self.device = device

        # TEST
        #self.compute_values(np.random.randn(obssize).reshape(1, -1))

    def compute_values(self, states):
        """
        compute value function for given states
        states: numpy array of size [numsamples, obssize]
        return: numpy array of size [numsamples]
        """
        if not torch.is_tensor(states): states = torch.FloatTensor(states).to(self.device)
        return self.model(states)

    def train(self, G, state_vals):
        """
        states: numpy array
        targets: numpy array
        """

        #calculate MSE loss
        val_loss = torch.nn.functional.mse_loss(state_vals, G)

        #Backpropagate
        self.optimizer.zero_grad()
        val_loss.backward()
        self.optimizer.step()

### Summary of pseudocode

The critical components of the pseudocode are as follows.

**Collect trajectories** Given current policy $\pi_\phi$, we can rollout using the policy by executing $a_t \sim \pi_\phi(s_t)$.

**Update value function** Value function update is based on minimizing the L2 loss between predicted value function and estimated value functions. For each state $s_i, i=0,\ldots, T$ in a trajectory of length $T+1$, compute $\hat{V}(s_i)$ as dicounted reward over the rest of the path (as defined above).  
Then take one gradient step to update $\theta$ using the gradient of the following loss:

$$\frac{1}{T+1} \sum_{i=0}^T(V_\theta(s_i) - \hat{V}(s_i))^2$$

For your conveience, below we have provided a function discounted_rewards(r,gamma) that takes as inputs a list of $T$ rewards $r$ and computes all discounted rewards $\hat V(s_i), i=0, 1, 2, \ldots, T$.

**Update policy using PG** To compute PG, we need to first monte carlo estimate action-value function $\hat{Q}(s_i,a_i)$. Given a trajectory with rewards $r=[r_0, r_1, r_2, \ldots, r_T]$, this can also be computed for all $s_i, a_i$ in this trajectory using the discounted_rewards(r, gamma) function below.

Then use value function as a baseline to compute advantage

$$\hat{A}(s_i,a_i) = \hat{Q}(s_i,a_i) - V_\theta(s_i)$$

Then compute surrogate loss

$$L = - \frac{1}{(T+1)}\sum_{i} \hat{A}(s_i,a_i) \log \pi(a_i|s_i) $$

The policy is updated by $$\phi \leftarrow \phi - \alpha  \nabla_\phi L \approx \phi + \alpha \nabla_\phi \rho(\pi_\phi)$$

In [ ]:
def discounted_rewards(r, gamma, device, normalize=True):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_sum = 0
    for i in reversed(range(0,len(r))):
        discounted_r[i] = running_sum * gamma + r[i]
        running_sum = discounted_r[i]

    if normalize:
      discounted_r = (discounted_r - discounted_r.mean()) / discounted_r.std()

    return torch.FloatTensor(discounted_r).to(device)

### Main implementation : Policy gradient algorithm

Combine all the above steps and implement the policy gradient algorithm with value function baseline in the cell below. The use of baseline is optional.

In [ ]:
import random
import os

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    env.seed(seed)

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
def get_action(policy, state, device):
    probs = policy.compute_prob(state)
    dist = torch.distributions.Categorical(probs)
    a = dist.sample()

    #return action
    return a.item(), dist.log_prob(a)

In [ ]:
#%%wandb
#remove the above line if you do not want to see wandb plots in your notebook. You can always see them on the wandb website.

#You can change the code in this cell anyway you want
#However, just make sure per epsiode reward during the run of this algorithm is being recorded in list rrecord
#and logged on wandb as in the last few lines.

# parameter initializations (you can change any of these)
alpha = 1e-2  # learning rate for PG
beta = 1e-2  # learning rate for baseline
iterations = 1000  # total num of iterations
max_episode_len = 10000
envname = "CartPole-v0"  # environment name
gamma = .99  # discount
opt = 'adam'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
normalize_discounted_rewards = True
seed = 0

# initialize environment
env = gym.make(envname)
obssize = env.observation_space.low.size
actsize = env.action_space.n
seed_everything(seed)

# initialize networks
seed_everything(seed)
actor = Policy(obssize, actsize, alpha, device)  # policy initialization: IMPORTANT: this is the policy you will be scored on
baseline = ValueFunction(obssize, beta, device)  # baseline initialization

run = wandb.init(
    # set the wandb project where this run will be logged
    project="rl-cartpole",

    # track hyperparameters and run metadata
    config={
    "method": "a2c",
    "alpha": alpha,
    "beta": beta,
    "iterations": iterations,
    "gamma": gamma,
    "device": device,
    "max_episode_len": max_episode_len,
    "seed": seed
    }
)

In [ ]:
#To record training reward for logging and plotting purposes
rrecord = []
for episode in tqdm_notebook(range(iterations)):
    state = env.reset()
    env.seed(0)

    env.action_space.seed(0)
    traj = []
    score = 0
    for step in range(max_episode_len):
        action, lp = get_action(actor, state, device)
        new_state, reward, done, _ = env.step(action)
        score += reward

        traj.append([state, action, reward, lp])

        if done:
            break
        state = new_state

    rrecord.append(score)

    # check if we are done training
    #printing moving averages for smoothed visualization.
    #Do not change below: this assume you recorded the sum of rewards in each episide in the list rrecord
    fixedWindow=100
    movingAverage=0
    if len(rrecord) >= fixedWindow:
        movingAverage=np.mean(rrecord[len(rrecord)-fixedWindow:len(rrecord)-1])
        if movingAverage >= 195: # env is solved
          break
    #wandb logging
    wandb.log({ "training reward" : rrecord[-1], "training reward moving average" : movingAverage})

    states = [step[0] for step in traj]
    actions = [step[1] for step in traj]
    rewards = [step[2] for step in traj]
    lps = [step[3] for step in traj]


    G = discounted_rewards(rewards, gamma, device)

    state_vals = []
    for state in states:
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        state_vals.append(baseline.model(state))

    state_vals = torch.stack(state_vals).squeeze()

    baseline.train(G, state_vals)
    ads = [g - v for g, v in zip(G, state_vals)]
    ads = torch.tensor(ads).to(device)
    actor.train(ads, lps)

<ipython-input-23-19e610cd24ff>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for episode in tqdm_notebook(range(iterations)):


  0%|          | 0/1000 [00:00<?, ?it/s]

Finally, we evaluate the performance of the trained agent. We will evaluate the performance of the trained policy. The evaluation will be run for 100 epsiodes and print out the average performance across these episodes. Please **do not** change the code below.

In [ ]:
# DO NOT CHANGE CODE HERE

### DO NOT CHANGE
def evaluate(policy, env, episodes):
    # main iteration
    score = 0
    for episode in range(episodes):

        obs = env.reset()
        done = False
        rsum = 0

        while not done:

            p = policy.compute_prob(np.expand_dims(obs,0)).cpu().data.numpy().ravel()
            p /= np.sum(p)
            action = np.random.choice(np.arange(2), size=1, p=p)[0]

            # env stepping forward
            newobs, r, done, _ = env.step(action)

            # update data
            rsum += r
            obs = newobs


        wandb.log({"eval reward" : rsum})
        score +=rsum
    score = score/episodes

    return score

In [ ]:
# DO NOT CHANGE CODE HERE
# after training, we will evaluate the performance of the learned policy "actor"
# on a target environment
env_test = gym.make(envname)
eval_episodes = 1000
score = evaluate(actor, env_test, eval_episodes)
wandb.run.summary["score"]=score
print("eval performance of the learned policy: {}".format(score))

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


eval performance of the learned policy: 22.735


In [ ]:
run.finish()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


eval reward,▁█▃▁▃█▁▂▂▁▁▆███▁█▃███▅▁▁▁█▁▁▁▆▆▁▂█▁██▁▅▁
training reward,▁▁▂▁▁▁▂▃▁▆▄▆▄▆████▅▅▄▄▅▆▆▆▆▇███▂▂███████
training reward moving average,▁▁▁▁▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇▇▇▇▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇█
eval reward,200
score,198
training reward,200.0
training reward moving average,194.52525
